In [1]:
import pandas as pd
import os
import geopy.distance
import numpy as np
from pathlib import Path

In [10]:
bus_station = pd.read_csv('tpe_bus_station_info_after_api_call.csv')
mrt_station = pd.read_csv('mrt_station.csv')
bike_station = pd.read_csv('TPE_bike_station.csv')

In [11]:
bike_station['district_tw'] = bike_station['district_tw'].replace('臺大公館校區','大安區')

In [12]:
bike_station.rename(
    {
        'name_tw':'station_name',
        'district_tw':'district',
        'lat':'lat',
        'lng':'lng'

    }
    ,axis=1,
    inplace=True
                    )

In [17]:

# bike_station['station_name'] = bike_station['station_name'].apply(lambda x: x.split('_')[1])
bike_station = bike_station[['station_no','station_name',
                             'city_code','district','lng','lat']]

In [18]:
mrt_station.rename(
    {
        'StationPosition.PositionLon':'lng',
        'StationPosition.PositionLat':'lat'
    },
    axis=1,
    inplace=True
)

In [19]:
mrt_dist = []
bus_dist = []
for idx, row in bike_station.iterrows():
    point_1 = row[['lat','lng']].to_numpy()
    mrt_dist.append(int(mrt_station[['lat','lng']].apply(lambda row:
                                 geopy.distance.geodesic(point_1,row.to_numpy()).m,
                                 axis=1).min()))
    bus_dist.append(int(bus_station[['lat','lng']].apply(lambda row:
                                 geopy.distance.geodesic(point_1,row.to_numpy()).m,
                                 axis=1).min()))
bike_station['near_to_bus'] = bus_dist
bike_station['near_to_mrt'] = mrt_dist
del mrt_station,bus_station

/var/folders/w3/7pcg5gg91fzfkchg8ds2t6w00000gp/T/ipykernel_35104/208401987.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_station['near_to_bus'] = bus_dist


In [26]:
ubike_rt = pd.read_csv('ubike_0501.csv')
ubike_rt.drop_duplicates(inplace=True)
ubike_rt['sarea'] = ubike_rt['sarea'].replace('臺大公館校區','大安區')
ubike_rt['sna'] = ubike_rt['sna'].apply(lambda x: x.split('_')[1])
ubike_rt = ubike_rt[ubike_rt['act']==1]

In [27]:
ubike_rt['thres'] = np.min(np.column_stack(((ubike_rt['tot']*0.1).to_numpy(),np.full(len(ubike_rt),5))),axis=1)
def almost_empty(row):
    bike = row['sbi']
    thres = row['thres']
    if bike <thres:
    # if bike <2:
        return True
    else:
        return False
    
ubike_rt['almost_empty'] = ubike_rt.apply(almost_empty,axis=1)
ubike_rt['date'] = pd.to_datetime(ubike_rt['updateTime']).dt.date
ubike_rt['hour'] = pd.to_datetime(ubike_rt['updateTime']).dt.hour


In [24]:
bike_station.head()

,station_no,station_name,city_code,district,lng,lat,near_to_bus,near_to_mrt
0,500101001,捷運科技大樓站,TPE,大安區,121.54360,25.02605,7,15
1,500101002,復興南路二段273號前,TPE,大安區,121.54357,25.02565,37,53
2,500101003,國北教大實小東側門,TPE,大安區,121.54124,25.02429,122,302
3,500101004,和平公園東側,TPE,大安區,121.54282,25.02351,79,296
4,500101005,辛亥復興路口西北側,TPE,大安區,121.54299,25.02153,80,510


In [36]:
result = ubike_rt.groupby(['sna','date','hour'],as_index=False).agg({'almost_empty':'mean'})

In [38]:
result = result.merge(bike_station,left_on='sna',right_on='station_name',how='inner')

In [39]:
result.columns

Index(['sna', 'date', 'hour', 'almost_empty', 'station_no', 'station_name',
       'city_code', 'district', 'lng', 'lat', 'near_to_bus', 'near_to_mrt'],
      dtype='object')

In [40]:
result = result[['station_name','near_to_mrt','near_to_bus','date','hour','lng','lat','almost_empty']]

In [41]:
result.to_csv('pipeline_F4_result.csv',index=False)

In [42]:
result.shape

(430171, 8)